# Feature Importance Analysis & Model Selection
## TASK 6: Model Selection - Feature Analysis & Final Model Choice

**Date:** December 30, 2025  
**Purpose:** Analyze feature importance and select best model  
**Based on:** Cross-validation results

---
## Setup

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print('✓ Libraries loaded')

---
## Part 1: Load Cross-Validation Results

In [ ]:
# Load CV results
cv_results = pd.read_csv('cross_validation_results.csv')

print('Cross-Validation Results:')
print('='*80)
print(cv_results.to_string(index=False))

# Identify best model
best_model_idx = cv_results['Accuracy_Mean'].idxmax()
best_model_name = cv_results.loc[best_model_idx, 'Model']
best_accuracy = cv_results.loc[best_model_idx, 'Accuracy_Mean']

print('\n' + '='*80)
print(f'🏆 BEST MODEL (by CV): {best_model_name}')
print(f'   Mean Accuracy: {best_accuracy:.4f}')
print('='*80)

---
## Part 2: Model Selection Criteria

In [ ]:
# Comprehensive model selection analysis
print('\n' + '='*80)
print('MODEL SELECTION ANALYSIS')
print('='*80)

# 1. Accuracy ranking
print('\n1. Accuracy Ranking:')
sorted_acc = cv_results.sort_values('Accuracy_Mean', ascending=False)
for i, (idx, row) in enumerate(sorted_acc.iterrows(), 1):
    print(f"   {i}. {row['Model']}: {row['Accuracy_Mean']:.4f} ± {row['Accuracy_Std']:.4f}")

# 2. Stability ranking (lowest std)
print('\n2. Stability Ranking (Lower std = More stable):')
sorted_std = cv_results.sort_values('Accuracy_Std')
for i, (idx, row) in enumerate(sorted_std.iterrows(), 1):
    stability = 'Excellent' if row['Accuracy_Std'] < 0.02 else 'Good' if row['Accuracy_Std'] < 0.05 else 'Fair'
    print(f"   {i}. {row['Model']}: {row['Accuracy_Std']:.4f} ({stability})")

# 3. F1-Score ranking
print('\n3. F1-Score Ranking:')
sorted_f1 = cv_results.sort_values('F1_Mean', ascending=False)
for i, (idx, row) in enumerate(sorted_f1.iterrows(), 1):
    print(f"   {i}. {row['Model']}: {row['F1_Mean']:.4f} ± {row['F1_Std']:.4f}")

# 4. Overall score (combination)
print('\n4. Combined Score (Accuracy - Std_penalty):')
cv_results['Combined_Score'] = cv_results['Accuracy_Mean'] - (cv_results['Accuracy_Std'] * 2)
sorted_combined = cv_results.sort_values('Combined_Score', ascending=False)
for i, (idx, row) in enumerate(sorted_combined.iterrows(), 1):
    print(f"   {i}. {row['Model']}: {row['Combined_Score']:.4f}")

In [ ]:
# Final model selection
print('\n' + '='*80)
print('FINAL MODEL SELECTION')
print('='*80)

selected_model_idx = sorted_combined.index[0]
selected_model = cv_results.loc[selected_model_idx]

print(f'\n✅ SELECTED MODEL: {selected_model["Model"]}')
print('\nSelection Rationale:')
print(f'  • Accuracy: {selected_model["Accuracy_Mean"]:.4f} ± {selected_model["Accuracy_Std"]:.4f}')
print(f'  • Precision: {selected_model["Precision_Mean"]:.4f} ± {selected_model["Precision_Std"]:.4f}')
print(f'  • Recall: {selected_model["Recall_Mean"]:.4f} ± {selected_model["Recall_Std"]:.4f}')
print(f'  • F1-Score: {selected_model["F1_Mean"]:.4f} ± {selected_model["F1_Std"]:.4f}')
print(f'  • Combined Score: {selected_model["Combined_Score"]:.4f}')

print('\nWhy this model?')
if selected_model['Accuracy_Mean'] == cv_results['Accuracy_Mean'].max():
    print('  ✓ Highest accuracy')
if selected_model['Accuracy_Std'] == cv_results['Accuracy_Std'].min():
    print('  ✓ Most stable (lowest variance)')
if selected_model['F1_Mean'] == cv_results['F1_Mean'].max():
    print('  ✓ Best F1-Score (balanced metric)')
if selected_model['Combined_Score'] == cv_results['Combined_Score'].max():
    print('  ✓ Best combined score (accuracy + stability)')

# Save selection
final_model_name = selected_model['Model']

---
## Part 3: Load Data for Feature Importance

In [ ]:
# Load data
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv').values.ravel()
y_test = pd.read_csv('y_test.csv').values.ravel()

# Combine for final training
X_full = pd.concat([X_train, X_test], axis=0, ignore_index=True)
y_full = np.concatenate([y_train, y_test])

feature_names = X_train.columns.tolist()

print('✓ Data loaded')
print(f'Total samples: {len(X_full)}')
print(f'Total features: {len(feature_names)}')

---
## Part 4: Feature Importance - All Models

In [ ]:
# Train all models on full data to get feature importance
print('Training models on full dataset for feature importance analysis...')

# Logistic Regression
lr_model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42, multi_class='ovr'))
])
lr_model.fit(X_full, y_full)
print('✓ Logistic Regression trained')

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_full, y_full)
print('✓ Random Forest trained')

# XGBoost
xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1,
                              objective='multi:softprob', random_state=42, n_jobs=-1)
xgb_model.fit(X_full, y_full)
print('✓ XGBoost trained')

---
## Part 5: Logistic Regression - Coefficient Analysis

In [ ]:
# Logistic Regression coefficients (averaged across classes)
lr_classifier = lr_model.named_steps['classifier']
lr_coef = np.abs(lr_classifier.coef_).mean(axis=0)

lr_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': lr_coef
}).sort_values('Importance', ascending=False)

print('\n' + '='*80)
print('LOGISTIC REGRESSION - TOP 15 FEATURES (by coefficient magnitude)')
print('='*80)
print(lr_importance.head(15).to_string(index=False))

lr_importance.to_csv('feature_importance_logistic_regression_final.csv', index=False)
print('\n✓ Saved: feature_importance_logistic_regression_final.csv')

---
## Part 6: Random Forest - Feature Importance

In [ ]:
# Random Forest feature importance
rf_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print('\n' + '='*80)
print('RANDOM FOREST - TOP 15 FEATURES')
print('='*80)
print(rf_importance.head(15).to_string(index=False))

rf_importance.to_csv('feature_importance_random_forest_final.csv', index=False)
print('\n✓ Saved: feature_importance_random_forest_final.csv')

---
## Part 7: XGBoost - Feature Importance

In [ ]:
# XGBoost feature importance (gain-based)
importance_dict = xgb_model.get_booster().get_score(importance_type='gain')
xgb_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': [importance_dict.get(f'f{i}', 0) for i in range(len(feature_names))]
}).sort_values('Importance', ascending=False)

print('\n' + '='*80)
print('XGBOOST - TOP 15 FEATURES (by gain)')
print('='*80)
print(xgb_importance.head(15).to_string(index=False))

xgb_importance.to_csv('feature_importance_xgboost_final.csv', index=False)
print('\n✓ Saved: feature_importance_xgboost_final.csv')

---
## Part 8: Compare Feature Importance Across Models

In [ ]:
# Normalize importance to 0-1 scale for comparison
lr_norm = (lr_importance['Importance'] - lr_importance['Importance'].min()) / \
          (lr_importance['Importance'].max() - lr_importance['Importance'].min())
rf_norm = (rf_importance['Importance'] - rf_importance['Importance'].min()) / \
          (rf_importance['Importance'].max() - rf_importance['Importance'].min())
xgb_norm = (xgb_importance['Importance'] - xgb_importance['Importance'].min()) / \
           (xgb_importance['Importance'].max() - xgb_importance['Importance'].min())

# Combine
comparison_df = pd.DataFrame({
    'Feature': feature_names,
    'LR_Importance': lr_importance.set_index('Feature').loc[feature_names, 'Importance'].values,
    'RF_Importance': rf_importance.set_index('Feature').loc[feature_names, 'Importance'].values,
    'XGB_Importance': xgb_importance.set_index('Feature').loc[feature_names, 'Importance'].values
})

# Calculate average importance
comparison_df['Average_Importance'] = comparison_df[['LR_Importance', 'RF_Importance', 'XGB_Importance']].mean(axis=1)
comparison_df = comparison_df.sort_values('Average_Importance', ascending=False)

print('\n' + '='*80)
print('FEATURE IMPORTANCE - CONSENSUS ACROSS ALL MODELS (Top 15)')
print('='*80)
print(comparison_df.head(15).to_string(index=False))

comparison_df.to_csv('feature_importance_comparison.csv', index=False)
print('\n✓ Saved: feature_importance_comparison.csv')

---
## Part 9: Visualize Feature Importance - Selected Model

In [ ]:
# Get importance for selected model
if final_model_name == 'Logistic Regression':
    selected_importance = lr_importance
    color = 'Blues'
elif final_model_name == 'Random Forest':
    selected_importance = rf_importance
    color = 'Greens'
else:  # XGBoost
    selected_importance = xgb_importance
    color = 'Oranges'

# Plot top 20 features
top_features = selected_importance.head(20)

plt.figure(figsize=(12, 10))
colors_plot = plt.cm.get_cmap(color)(np.linspace(0.4, 0.8, len(top_features)))
plt.barh(range(len(top_features)), top_features['Importance'].values, color=colors_plot)
plt.yticks(range(len(top_features)), top_features['Feature'].values)
plt.xlabel('Importance', fontsize=12, fontweight='bold')
plt.ylabel('Features', fontsize=12, fontweight='bold')
plt.title(f'Top 20 Feature Importance - {final_model_name} (Selected Model)', 
         fontsize=14, fontweight='bold', pad=15)
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('feature_importance_selected_model.png', dpi=300, bbox_inches='tight')
print('✓ Saved: feature_importance_selected_model.png')
plt.show()

---
## Part 10: Compare Top Features Across All Models

In [ ]:
# Plot comparison of top 15 features across models
top_consensus = comparison_df.head(15)

fig, ax = plt.subplots(figsize=(14, 8))

x = np.arange(len(top_consensus))
width = 0.25

# Normalize for fair comparison
lr_vals = (top_consensus['LR_Importance'] / top_consensus['LR_Importance'].max()).values
rf_vals = (top_consensus['RF_Importance'] / top_consensus['RF_Importance'].max()).values
xgb_vals = (top_consensus['XGB_Importance'] / top_consensus['XGB_Importance'].max()).values

ax.barh(x - width, lr_vals, width, label='Logistic Regression', color='#3498db', alpha=0.8)
ax.barh(x, rf_vals, width, label='Random Forest', color='#2ecc71', alpha=0.8)
ax.barh(x + width, xgb_vals, width, label='XGBoost', color='#e74c3c', alpha=0.8)

ax.set_yticks(x)
ax.set_yticklabels(top_consensus['Feature'].values)
ax.set_xlabel('Normalized Importance', fontsize=12, fontweight='bold')
ax.set_ylabel('Features', fontsize=12, fontweight='bold')
ax.set_title('Top 15 Features - Comparison Across All Models', 
            fontsize=14, fontweight='bold', pad=15)
ax.legend(loc='lower right', fontsize=10)
ax.grid(alpha=0.3, axis='x')
ax.invert_yaxis()

plt.tight_layout()
plt.savefig('feature_importance_all_models_comparison.png', dpi=300, bbox_inches='tight')
print('✓ Saved: feature_importance_all_models_comparison.png')
plt.show()

---
## Part 11: Feature Categories Analysis

In [ ]:
# Categorize features
def categorize_feature(feature_name):
    feature_lower = feature_name.lower()
    if 'rate' in feature_lower:
        return 'Rate Features'
    elif 'ratio' in feature_lower or 'remaining' in feature_lower:
        return 'Ratio/Remaining Features'
    elif 'interaction' in feature_lower:
        return 'Interaction Features'
    elif 'category' in feature_lower or 'flag' in feature_lower:
        return 'Categorical/Flag Features'
    else:
        return 'Original Features'

comparison_df['Category'] = comparison_df['Feature'].apply(categorize_feature)

# Category importance summary
category_summary = comparison_df.groupby('Category').agg({
    'Average_Importance': ['mean', 'sum', 'count']
}).round(4)

print('\n' + '='*80)
print('FEATURE CATEGORY ANALYSIS')
print('='*80)
print(category_summary)

print('\nInterpretation:')
print('  • Mean: Average importance per feature in category')
print('  • Sum: Total contribution of category')
print('  • Count: Number of features in category')

---
## Part 12: Key Insights & Recommendations

In [ ]:
print('\n' + '='*80)
print('KEY INSIGHTS & RECOMMENDATIONS')
print('='*80)

print(f'\n1. SELECTED MODEL: {final_model_name}')
print(f'   Performance: {selected_model["Accuracy_Mean"]:.4f} ± {selected_model["Accuracy_Std"]:.4f}')

print('\n2. TOP 5 MOST IMPORTANT FEATURES (Consensus):')
for i, (idx, row) in enumerate(comparison_df.head(5).iterrows(), 1):
    print(f'   {i}. {row["Feature"]} (Avg importance: {row["Average_Importance"]:.4f})')

print('\n3. FEATURE ENGINEERING IMPACT:')
engineered_count = (comparison_df['Category'] != 'Original Features').sum()
engineered_top10 = (comparison_df.head(10)['Category'] != 'Original Features').sum()
print(f'   • {engineered_count}/{len(comparison_df)} features are engineered')
print(f'   • {engineered_top10}/10 top features are engineered')
if engineered_top10 >= 7:
    print('   → Feature engineering was HIGHLY SUCCESSFUL')
elif engineered_top10 >= 4:
    print('   → Feature engineering was SUCCESSFUL')
else:
    print('   → Original features remain most important')

print('\n4. RECOMMENDATIONS:')
print(f'   ✓ Deploy {final_model_name} for production')
print(f'   ✓ Monitor top 10 features for data quality')
print(f'   ✓ Consider feature selection to reduce complexity')

# Check for low importance features
low_importance = comparison_df[comparison_df['Average_Importance'] < 0.001]
if len(low_importance) > 0:
    print(f'   ⚠ {len(low_importance)} features have very low importance')
    print(f'   → Consider removing: {low_importance["Feature"].head(5).tolist()}')

print('\n5. NEXT STEPS:')
print('   1. Hyperparameter tuning for selected model')
print('   2. Test on holdout set')
print('   3. Deploy model with monitoring')
print('   4. Regular retraining schedule')

print('\n' + '='*80)

---
## Part 13: Save Final Selection Report

In [ ]:
# Create final report
final_report = {
    'Selected_Model': final_model_name,
    'CV_Accuracy_Mean': selected_model['Accuracy_Mean'],
    'CV_Accuracy_Std': selected_model['Accuracy_Std'],
    'CV_F1_Mean': selected_model['F1_Mean'],
    'CV_F1_Std': selected_model['F1_Std'],
    'Top_Feature_1': comparison_df.iloc[0]['Feature'],
    'Top_Feature_2': comparison_df.iloc[1]['Feature'],
    'Top_Feature_3': comparison_df.iloc[2]['Feature'],
    'Total_Features': len(feature_names),
    'Selection_Date': '2025-12-30'
}

report_df = pd.DataFrame([final_report])
report_df.to_csv('final_model_selection_report.csv', index=False)
print('✓ Saved: final_model_selection_report.csv')

print('\n' + '='*80)
print('ALL FILES SAVED SUCCESSFULLY')
print('='*80)
print('\nGenerated files:')
print('  • feature_importance_logistic_regression_final.csv')
print('  • feature_importance_random_forest_final.csv')
print('  • feature_importance_xgboost_final.csv')
print('  • feature_importance_comparison.csv')
print('  • feature_importance_selected_model.png')
print('  • feature_importance_all_models_comparison.png')
print('  • final_model_selection_report.csv')

---
## Summary

### Model Selection Process:
1. **Cross-Validation Results:** Reviewed 5-fold CV performance
2. **Selection Criteria:** Accuracy, stability, F1-score, combined score
3. **Final Choice:** Selected best performing model

### Feature Importance Analysis:
- **Logistic Regression:** Coefficient magnitude (absolute values)
- **Random Forest:** Mean decrease in impurity
- **XGBoost:** Gain-based importance
- **Consensus:** Average across all models

### Key Findings:
- Identified top predictive features
- Evaluated feature engineering impact
- Found low-value features for potential removal
- Provided deployment recommendations

### Files Generated:
- Feature importance for each model (CSV)
- Cross-model comparison (CSV)
- Visualizations (PNG)
- Final selection report (CSV)

---
**Analysis Complete! Model selected for deployment.**